In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/16715/hopper-sim/scripts/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Installed METIS_jll ────────── v5.1.1+0
   Installed Ipopt ────────────── v0.9.0
   Installed MutableArithmetics ─ v0.3.1
   Installed ASL_jll ──────────── v0.1.3+0
   Installed BenchmarkTools ───── v1.2.2
   Installed MathOptInterface ─── v0.10.6
    Updating `/mnt/064AC6424AC62E6D/git_workspace/16715/hopper-sim/scripts/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
  [6a86dc24] + FiniteDiff v2.8.1
  [b6b21f68] + Ipopt v0.9.0
  [b8f27783] + MathOptInterface v0.10.6
    Updating `/mnt/064AC6424AC62E6D/git_workspace/16715/hopper-sim/scripts/Manifest.toml`
  [4fba245c] + ArrayInterface v3.2.1
  [6e4b80f9] + BenchmarkTools v1.2.2
  [b99e7846] + BinaryProvider v0.5.10
  [523fee87] + CodecBzip2 v0.7.2
  [944b1d66] + CodecZlib v0.7.0
  [34da2185] + Compat v3.40.0
  [6a86dc24] + FiniteDiff v2.8.1
  [615f187c] + IfElse v0.1.1
  [b

In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays
using SparseArrays
using ForwardDiff
const FD = ForwardDiff

WebIO._IJuliaInit()

ForwardDiff

In [3]:
# link lengths
const l0 = [0.1; 0; 0]
const l1 = [0.3; 0; 0]
const l2 = [0.3; 0; 0]
const l3 = 0.1
const l4 = 0.2
const l5 = 0.0205
const lc = [l3; 0; 0]
const lee = [l3 + l4; 0; l5] # sqrt((l3 + l4)^2 + l5^2)

# CoM locations
const l_cb = [0; 0.004; 0]
const l_c0 = [0.0125108364230515; 0.00117191218927888; 0]
const l_c1 = [0.149359714867044; 0; 0]
const l_c2 = [0.0469412900551914; 0; 0]
const l_c3 = [0.113177000131857; 0; -0.015332867880069]

# link masses
const mb = 7  # kg
const m0 = 0.24644240965487
const m1 = 0.0707939028219395
const m2 = 0.276735496985514
const m3 = 0.130824780046739
# const m = Diagonal([m0, m1, m2, m3])
    
# gravity, obviously
const g = 9.807

# mass moment of inertia in axis of rotation
const Ib = Array([0.0024241 5.252E-06 2.0733E-19; 
                  5.252E-06 0.0044176 -3.1153E-19; 
                  2.0733E-19 -3.1153E-19 0.0022481])

const I0 = Array([3.83120149546952E-05 1.46925714738609E-05 -8.60106401672571E-06;
                  1.46925714738609E-05 0.000172067745507247 1.0427260925207E-06;
                  -8.60106401672571E-06 1.0427260925207E-06 0.00014745218068435])

const I1 = Array([3.06999775886187E-06 7.91090301514898E-12 -1.43705963146176E-12;
                  7.91090301514898E-12 0.000147960574744097 1.30742394049546E-11;
                  -1.43705963146176E-12 1.30742394049546E-11 0.000147884231885009])

const I2 = Array([3.43038397803592E-05 -2.90339844227483E-07 6.18680397558952E-06;
                  -2.90339844227483E-07 0.000302324068012293 2.25016327583562E-08;
                  6.18680397558952E-06 2.25016327583562E-08 0.00028292376778719])

const I3 = Array([1.76996970020568E-05 -5.3695427116208E-07 7.62350214406387E-07;
                  -5.3695427116208E-07 0.000164188445564489 -2.77843753828047E-07;
                  7.62350214406387E-07 -2.77843753828047E-07 0.000160656046697151])

3×3 Matrix{Float64}:
  1.76997e-5  -5.36954e-7    7.6235e-7
 -5.36954e-7   0.000164188  -2.77844e-7
  7.6235e-7   -2.77844e-7    0.000160656

In [4]:
M̄ = [mb*I(3) zeros(3, 27)
     zeros(3,3) Ib zeros(3, 24)
     zeros(3,6) m0*I(3) zeros(3,21);
     zeros(3,9) I0 zeros(3,18);
     zeros(3,12) m1*I(3) zeros(3,15);
     zeros(3,15) I1 zeros(3, 12);
     zeros(3,18) m2*I(3) zeros(3, 9);
     zeros(3, 21) I2 zeros(3, 6);
     zeros(3, 24) m3*I(3) zeros(3, 3);
     zeros(3, 27) I3]

30×30 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⠑⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢱⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢱⣶⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛

In [5]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1.0; -1.0; -1.0])

function G(Q)
    return L(Q)*H  # 4x3
end

function Ḡ(q)
    Qb = q[4:7]
    Q0 = q[11:14]
    Q1 = q[18:21]
    Q2 = q[25:28]
    Q3 = q[32:35]
    
    # 35x30
    return blockdiag(sparse(I, 3, 3), sparse(G(Qb)), 
                     sparse(I, 3, 3), sparse(G(Q0)), 
                     sparse(I, 3, 3), sparse(G(Q1)),
                     sparse(I, 3, 3), sparse(G(Q2)),
                     sparse(I, 3, 3), sparse(G(Q3)),)
end

Ḡ (generic function with 1 method)

In [6]:
function Expq(ϕ)
    
    # The quaternion exponential map ϕ → q 
    q = zeros(4)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
    
    return q
end

function rotate(Q, p)
    # Rotate a position vector p by a quaternion Q
    return H'L(Q)*R(Q)'*H*p
end

rotate (generic function with 1 method)

In [7]:
#Initial Conditions
rb = [0.0; 0.0; 0.5]  # start 0.5 meters above the ground
Qb = [1.0; 0; 0; 0]
q0 = 30*pi/180;
q1 = 120*(pi/180)
q2 = 150*(pi/180)
q3 = -120*(pi/180)

Tf = 1
h = 0.01
thist = 0:h:Tf
N = length(thist)

pb = rb + rotate(Qb, l_cb)  # position vector from world frame to *JOINTS* 0 and 2

Qb0 = Expq([0, q0, 0])  # quaternion from base to link 0
Q0 = L(Qb)*Qb0  # quaternion from world frame to link 0
r0 = pb + rotate(Q0, l_c0)

Q01 = Expq([0, q1, 0])  # quaternion from link 0 to link 1
Q1 = L(Q0)*Q01  # quaternion from world frame to link 1
r1 = pb + rotate(Q0, l0) + rotate(Q1, l_c1)  

Qb2 = Expq([0, q2, 0])  # quaternion from base to link 2
Q2 = L(Qb)*Qb2  # quaternion from world frame to link 2
r2 = pb + rotate(Q2, l_c2)

Q23 = Expq([0, q3, 0])  # quaternion from base to link 2
Q3 = L(Q2)*Q23  # quaternion from world frame to link 2
r3 = pb + rotate(Q2, l2) + rotate(Q3, l_c3)  

rb_0 = rb
r0_0 = r0
r1_0 = r1
r2_0 = r2
r3_0 = r3
Qb_0 = Qb
Q0_0 = Q0
Q1_0 = Q1
Q2_0 = Q2
Q3_0 = Q3

q_0 = [rb_0; Qb_0; r0_0; Q0_0; r1_0; Q1_0; r2_0; Q2_0; r3_0; Q3_0]
# @show c(q_0)


35-element Vector{Float64}:
  0.0
  0.0
  0.5
  1.0
  0.0
  0.0
  0.0
  0.010834702164954238
  0.00517191218927888
  0.49374458178847425
  0.9659258262890683
  0.0
  0.25881904510252074
  ⋮
  0.4765293549724043
  0.25881904510252074
  0.0
  0.9659258262890683
  0.0
 -0.16945989783706322
  0.004
  0.2801328468370613
  0.9659258262890683
  0.0
  0.2588190451025209
  0.0

In [8]:
#Torque input at joints
uhist = repeat([0 0 0 0 0]*1e-4, N)'
uhist[:, 1] = [0 0 0 0 0]
uhist[:, 2] = [0 0 0 0 0]
# 1 -> joint0
# 2 -> joint1
# 3 -> joint2
# 4 -> joint3
# 5 -> joint4 (parallel constraint)
#Corresponding F
Fhist = zeros(30,N)  # wrench [xyz force, xyz torque]
for k = 1:N
    Fhist[:,k] = [zeros(4); -uhist[1, k]-uhist[3, k]; 0; # body
                  zeros(4); uhist[1, k]-uhist[2, k]; 0; # link0
                  zeros(4); uhist[2, k]+uhist[5, k]; 0; # link1
                  zeros(4); uhist[3, k]-uhist[4, k]; 0; # link2
                  zeros(4); uhist[4, k]-uhist[5, k]; 0] # link3
end

In [9]:
function exert_wrench(u, pos,  b_orient, target)
    
end

exert_wrench (generic function with 1 method)

In [10]:
# make gravity zero for first two timesteps, see if that works
ghist = repeat([0 0 g], N)'
ghist[:, 1] = [0 0 0]
ghist[:, 2] = [0 0 0]

1×3 Matrix{Int64}:
 0  0  0

Solve the following optimization problem at every timestep. 
$$ \begin{align}
\min_{q_{k+1},\lambda_k,s_k} && \sum_{i=1}^8s_{k,i} \\
\text{st} && DEL(q_{k-1},q_k,q_{k+1}) &= 0 && \text{discrete Euler-Lagrange}\\ 
 && \|q_{k+1}\|^2 - 1 &= 0 && \text{quaternion norm constraint}\\ 
 && d(q_{k+1}) &\geq 0 && \text{foot must stay above ground}\\ 
 && s_k - \text{diag}(\lambda_k)d(q_{k+1}) &\geq 0 && \text{collision constraint}\\
 && \lambda_k &\geq 0 \\ 
 && s_k &\geq 0
\end{align} $$

We will be using IPOPT to solve this, where IPOPT takes problems of the following form
$$ \begin{align}
\min_x && f(x) \\ 
\text{st} && c_L \leq c(x) &\leq c_U && \text{constraint function with constraint bounds} \\ 
&& x_L \leq x &\leq x_U && \text{primal bounds}
\end{align}$$

In [11]:
function con(q)
    # joint constraint function
    # c(q_0) should be all zeros
    c_ = zeros(eltype(q), 24)
    
    rb = q[1:3]
    Qb = q[4:7]
    r0 = q[8:10]
    Q0 = q[11:14]
    r1 = q[15:17]
    Q1 = q[18:21]
    r2 = q[22:24]
    Q2 = q[25:28]
    r3 = q[29:31]
    Q3 = q[32:35]
    
    rf = r3 + rotate(Q3, lee-l_c3) # position of foot

    pb = rb + rotate(Qb, l_cb) # position vector from world frame to *JOINTS* 0 and 2
    
    c_[1:3] = pb - r0 - rotate(Q0, -l_c0)
    c_[4:5] = [0 1 0 0; 0 0 0 1]*L(Qb)'*Q0  # y axis rotation constraint
    c_[6:8] = r0 + rotate(Q0,  l0-l_c0) - r1 - rotate(Q1, -l_c1)
    c_[9:10] = [0 1 0 0; 0 0 0 1]*L(Q0)'*Q1
    c_[11:13] = pb - r2 - rotate(Q2, -l_c2)
    c_[14:15] = [0 1 0 0; 0 0 0 1]*L(Qb)'*Q2
    c_[16:18] = r2 + rotate(Q2, l2 - l_c2) - r3 - rotate(Q3, -l_c3)
    c_[19:20] = [0 1 0 0; 0 0 0 1]*L(Q2)'*Q3
    c_[21:23] = r1 + rotate(Q1, l1 - l_c1) - r3 - rotate(Q3, lc-l_c3)
    c_[24] = rf[3] - 0.025  # subtract radius of foot
             
    return c_
end

function Dc(q)
    ForwardDiff.jacobian(dq->con(dq),q)*Ḡ(q)
end

Dc (generic function with 1 method)

In [50]:
function del(m, I, r1, r2, r3, Q1, Q2, Q3, grav)
    [(1/h)*m*(r2-r1) - (1/h)*m*(r3-r2) - m*grav*h;
     (2.0/h)*G(Q2)'*L(Q1)*H*I*H'*L(Q1)'*Q2 + (2.0/h)*G(Q2)'*T*R(Q3)'*H*I*H'*L(Q2)'*Q3]
end
    
    
function DEL(q_1,q_2,q_3,λ,F1,F2, grav)
    
    rb_1 = q_1[1:3]
    Qb_1 = q_1[4:7]
    r0_1 = q_1[8:10]
    Q0_1 = q_1[11:14]
    r1_1 = q_1[15:17]
    Q1_1 = q_1[18:21]
    r2_1 = q_1[22:24]
    Q2_1 = q_1[25:28]
    r3_1 = q_1[29:31]
    Q3_1 = q_1[32:35]
    
    rb_2 = q_2[1:3]
    Qb_2 = q_2[4:7]
    r0_2 = q_2[8:10]
    Q0_2 = q_2[11:14]
    r1_2 = q_2[15:17]
    Q1_2 = q_2[18:21]
    r2_2 = q_2[22:24]
    Q2_2 = q_2[25:28]
    r3_2 = q_2[29:31]
    Q3_2 = q_2[32:35]
    
    rb_3 = q_3[1:3]
    Qb_3 = q_3[4:7]
    r0_3 = q_3[8:10]
    Q0_3 = q_3[11:14]
    r1_3 = q_3[15:17]
    Q1_3 = q_3[18:21]
    r2_3 = q_3[22:24]
    Q2_3 = q_3[25:28]
    r3_3 = q_3[29:31]
    Q3_3 = q_3[32:35]

    del1 = [del(mb, Ib, rb_1, rb_2, rb_3, Qb_1, Qb_2, Qb_3, grav);
            del(m0, I0, r0_1, r0_2, r0_3, Q0_1, Q0_2, Q0_3, grav);
            del(m1, I1, r1_1, r1_2, r1_3, Q1_1, Q1_2, Q1_3, grav);
            del(m2, I2, r2_1, r2_2, r2_3, Q2_1, Q2_2, Q2_3, grav);
            del(m3, I3, r3_1, r3_2, r3_3, Q3_1, Q3_2, Q3_3, grav)] 

    return del1 + (h/2.0)*F1 + (h/2.0)*F2 + reshape(h*λ'*Dc(q_2), 30)
end

DEL (generic function with 1 method)

In [51]:
function Dq3DEL(q_1,q_2,q_3,λ,F1,F2, grav)
    # @show Ḡ(q_3)
    ForwardDiff.jacobian(dq->DEL(q_1,q_2,dq,λ,F1,F2), q_3)*Ḡ(q_3)
end

Dq3DEL (generic function with 1 method)

In [52]:
using Ipopt
using MathOptInterface
const MOI = MathOptInterface

#Boilerplate setup code to interface with IPOPT.

struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)

    c_u = zeros(m)
    c_u[idx_ineq] .= Inf
    
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    objective(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,objective,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    constraint!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), constraint!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function ipopt_solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=10000, print=0)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol
    
    #Uncomment the following line to turn off verbose IPOPT output
    solver.options["print_level"] = print

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

ipopt_solve (generic function with 1 method)

In [53]:
@show n_c = size(con(q_0))[1]  # number of constraint rows, 24
@show n_q = size(q_0)[1]  # number of q rows, 35

n_c = (size(con(q_0)))[1] = 24
n_q = (size(q_0))[1] = 35


35

In [54]:
#Objective and constraint functions for IPOPT

function objective(z)
    qn = z[1:n_q]
    λ = z[n_q+1:n_q+n_c]
    s = z[n_q+1+n_c:n_q+n_c+n_c]
    
    return sum(s) #Minimize slacks associated with complementarity conditions
end

function constraint!(c,z)
    qn = z[1:n_q]
    λ = z[n_q+1:n_q+n_c]
    s = z[n_q+1+n_c:n_q+n_c+n_c]

    # nonlinear DEL equation                (30 equality constraint)
    # quaternion norm squared - 1 = 0       (5 equality constraints)
    # joint constraints                     (23 equality constraints)  c7
    # foot height must stay above ground    (1 inequality constraint)  c7
    # collision constraint                  (24 inequality constraint)

    c1 = DEL(qhist[:,k-1], qhist[:,k], qn, λ, Fhist[:,k-1],Fhist[:,k], ghist[:, k])  # 30x1
    c2 = norm(qn[4:7])^2 - 1  # 1x1
    c3 = norm(qn[11:14])^2 - 1
    c4 = norm(qn[18:21])^2 - 1
    c5 = norm(qn[25:28])^2 - 1
    c6 = norm(qn[32:35])^2 - 1
    
    c7 = con(qn)  # 24x1
    c8 = s - Diagonal(λ)*con(qn)  # 24x1
    c .= [c1; c2; c3; c4; c5; c6; c7; c8]

    return nothing
end

#Specify the indicies of c (constraint output) that should be non-negative.
#The rest will be treated as equality constraints.
#This can vary depending on how you stacked up c above.

nonnegative_constraint_indices = (58+1:58+25) # update this

function primal_bounds(n)
    #Enforce simple bound constraints on the decision variables (e.g. positivity) here
    # x_l ≤ [q; λ; s] ≤ x_u

    x_l = zeros(n)
    x_u = Inf*ones(n)
    x_l[1:35] = -Inf*ones(35)

    return x_l, x_u
end

primal_bounds (generic function with 1 method)

In [66]:
function constraint_check(z)
    qn = z[1:n_q]
    λ = z[n_q+1:n_q+n_c]
    s = z[n_q+1+n_c:n_q+n_c+n_c]

    c1 = DEL(qhist[:,k-1], qhist[:,k], qn, λ, Fhist[:,k-1],Fhist[:,k], ghist[:, k])  # 30x1
    c2 = norm(qn[4:7])^2 - 1  # 1x1
    c3 = norm(qn[11:14])^2 - 1
    c4 = norm(qn[18:21])^2 - 1
    c5 = norm(qn[25:28])^2 - 1
    c6 = norm(qn[32:35])^2 - 1
    
    c7 = con(qn)  # 24x1
    c8 = s - Diagonal(λ)*con(qn)  # 24x1
    A = [c1; c2; c3; c4; c5; c6; c7[1:23]]
    B = [c7[24]; c8]
    if !isapprox(A, zeros(58); atol=1e-6, rtol=0)
        e = 1
        print("\n", A, "\n")
        print(findall(A .< -ones(58)*1e-6), " is less than 0 \n")
        print(findall(A .> ones(58)*1e-6), " is greater than 0 \n")
        
    elseif B < -ones(25)*1e-6
        e = 1
        print("\n", B, "\n")
        print(findall(B .< -ones(25)*1e-6))
    else
        e = 0
    end
    
    return e
end

constraint_check (generic function with 2 methods)

In [67]:
#Solve with IPOPT
n_nlp = n_q + n_c + n_c  # size of decision variables
m_nlp = 83  # size of constraint! output 
nlp_prob = ProblemMOI(n_nlp,m_nlp, idx_ineq=nonnegative_constraint_indices);

#Initial conditions
qhist = zeros(n_q,N)
qhist[:,1] .= q_0
qhist[:,2] .= q_0  # this may need to be fixed

qhist[4:7,2] .= qhist[4:7,2]/norm(qhist[4:7,2])
qhist[11:14,2] .= qhist[11:14,2]/norm(qhist[11:14,2])
qhist[18:21,2] .= qhist[18:21,2]/norm(qhist[18:21,2])
qhist[25:28,2] .= qhist[25:28,2]/norm(qhist[25:28,2])
qhist[32:35,2] .= qhist[32:35,2]/norm(qhist[32:35,2])

λhist = zeros(n_c,N-1)
shist = zeros(n_c,N-1)
k = 0

for kk = 2:(N-1)
    print("Simulation ", kk/(N-1)*100, " % complete \n")
    flush(stdout)
    k = kk
    z_guess = [qhist[:,k]; zeros(n_q); ones(n_q)]
    z_sol = ipopt_solve(z_guess, nlp_prob, print=0);
    qhist[:,k+1] .= z_sol[1:n_q]
    λhist[:,k] .= z_sol[n_q + 1:n_q + n_c]
    shist[:,k] .= z_sol[n_q + 1 + n_c:n_q + n_c + n_c]
    e = constraint_check(z_sol)
    # print("\n", e, "\n")
    if e == true
        print("\n Sim stopped due to ipopt infeasibility \n")
        break
    end
    
end

Simulation 2.0 % complete 
Simulation 3.0 % complete 
Simulation 4.0 % complete 
Simulation 5.0 % complete 
Simulation 6.0 % complete 
Simulation 7.000000000000001 % complete 
Simulation 8.0 % complete 
Simulation 9.0 % complete 
Simulation 10.0 % complete 
Simulation 11.0 % complete 
Simulation 12.0 % complete 
Simulation 13.0 % complete 
Simulation 14.000000000000002 % complete 
Simulation 15.0 % complete 
Simulation 16.0 % complete 
Simulation 17.0 % complete 
Simulation 18.0 % complete 
Simulation 19.0 % complete 
Simulation 20.0 % complete 
Simulation 21.0 % complete 
Simulation 22.0 % complete 

[7.549200847778437e-11, 1.5033731193971158e-10, 5.1924992083840493e-11, -2.4032767278797575e-11, -1.4268442118264072e-15, -1.0402259586881788e-10, -7.525084505174973e-11, -6.003345101495073e-11, -5.1927234864454497e-11, -1.5429893150969867e-10, -0.00015362197430357526, 7.832222934327891e-6, 9.597580922279109e-11, 9.875180242610424e-11, 9.754498315854565e-11, -8.504219502253581e-8, 1.09514

In [18]:
qhist

35×101 Matrix{Float64}:
  0.0          0.0          3.34109e-14  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0         -6.86478e-16     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.5          0.5          0.5             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0          1.0          1.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0         -2.76568e-15     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0          3.28955e-18  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0         -8.22728e-14     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0108347    0.0108347    0.0108347       0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.00517191   0.00517191   0.00517191      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.493745     0.493745     0.493745        0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.965926     0.965926     0.965926     …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0          1.86223e-14     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.258819     0.258819     0.258819    

In [19]:
using MeshCat, GeometryBasics, CoordinateTransformations, Rotations, Plots #, GeometryTypes

In [20]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [23]:
# Zoom in--the robot falls through floor since there is no contact

delete!(vis)

green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.8))
red_material = MeshPhongMaterial(color=RGBA(1, 0, 0, 0.8))

#l_345 = sqrt((l3 + l4)^2 + l5^2)
# sphere = Sphere(Point(0, 0, 0), 3)
# rect = Rectangle(0.2, 0.2, 0.2, 0.2)
cylinderb = Cylinder(Point(0, -0.02, l_cb[2]), Point(0, 0.02, l_cb[2]), 0.04)  #float(l_cb)
cylinder0 = Cylinder(Point(-l_c0[1], -l_c0[2], -l_c0[3]), Point((l0-l_c0)[1], (l0-l_c0)[2], (l0-l_c0)[3]), 0.008)
cylinder1 = Cylinder(Point(-l_c1[1], -l_c1[2], -l_c1[3]), Point((l1-l_c1)[1],(l1-l_c1)[2],(l1-l_c1)[2]), 0.008)
cylinder2 = Cylinder(Point(-l_c2[1], -l_c2[2], -l_c2[3]), Point((l2-l_c2)[1], (l2-l_c2)[2], (l2-l_c2)[3]), 0.008)
cylinder3 = Cylinder(Point(-l_c3[1], -l_c3[2], -l_c3[3]), Point((lee-l_c3)[1], (lee-l_c3)[2],  (lee-l_c3)[3]), 0.008)

# setobject!(vis["sphere"],sphere,red_material)
setobject!(vis["cylinderb"],cylinderb,red_material)
setobject!(vis["cylinder0"],cylinder0,green_material)
setobject!(vis["cylinder1"],cylinder1,green_material)
setobject!(vis["cylinder2"],cylinder2,green_material)
setobject!(vis["cylinder3"],cylinder3,green_material)

for k = 1:N
    
    q = qhist[:, k]
    # set position and attitude
    
    positionb = Translation(q[1:3]...)
    attitudeb = LinearMap(UnitQuaternion(q[4:7]))
    position0 = Translation(q[8:10]...)
    attitude0 = LinearMap(UnitQuaternion(q[11:14]))
    position1 = Translation(q[15:17]...)
    attitude1 = LinearMap(UnitQuaternion(q[18:21]))
    position2 = Translation(q[22:24]...)
    attitude2 = LinearMap(UnitQuaternion(q[25:28]))
    position3 = Translation(q[29:31]...)
    attitude3 = LinearMap(UnitQuaternion(q[32:35]))

    settransform!(vis["cylinderb"], compose(positionb,attitudeb))
    settransform!(vis["cylinder0"], compose(position0,attitude0))
    settransform!(vis["cylinder1"], compose(position1,attitude1))
    settransform!(vis["cylinder2"], compose(position2,attitude2))
    settransform!(vis["cylinder3"], compose(position3,attitude3))
    sleep(0.1)
end